**Linear Regression**
Model Linear Regression memprediksi nilai dengan menjumlahkan fitur input yang telah diberi bobot, ditambah dengan *bias term* (atau *intercept term*). Persamaan prediksinya adalah $\hat{y}=\theta_{0}+\theta_{1}x_{1}+\theta_{2}x_{2}+\cdot\cdot\cdot+\theta_{n}x_{n}$. Dalam bentuk vektor, ini menjadi $\hat{y}=h_{\theta}(x)=\theta\cdot x$.

Untuk melatih model Linear Regression, tujuannya adalah menemukan parameter $\theta$ yang meminimalkan *Mean Squared Error* (MSE). Fungsi biaya MSE untuk Linear Regression adalah $MSE(X,h_{\theta})=\frac{1}{m}\sum_{i=1}^{m}(\theta^{T}x^{(i)}-y^{(i)})^{2}$.

Ada dua cara utama untuk melatih model Linear Regression:

1.  **Normal Equation**: Ini adalah solusi *closed-form* yang secara langsung menghitung parameter model yang meminimalkan fungsi biaya. Persamaannya adalah $\hat{\theta}=(X^{\top}X)^{-1}X^{\top}y$.
    * **Contoh Kode**:
        ```python
        import numpy as np

        X = 2 * np.random.rand(100, 1)
        y = 4 + 3 * X + np.random.randn(100, 1)

        X_b = np.c_[np.ones((100, 1)), X] # Menambahkan x0=1 ke setiap instance
        theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
        print(theta_best)
        ```
        Kode ini menghasilkan data linear, menambahkan *bias term* ($x_0=1$) ke setiap *instance* X, dan kemudian menghitung `theta_best` (parameter optimal) menggunakan Normal Equation. Output `theta_best` akan mendekati `[[4.], [3.]]` meskipun ada *noise*.

    * **Kompleksitas Komputasi**: Menghitung invers dari matriks $X^TX$ (ukuran $(n+1) \times (n+1)$) memiliki kompleksitas sekitar $O(n^{2.4})$ hingga $O(n^3)$. Ini menjadi sangat lambat ketika jumlah fitur ($n$) sangat besar (misalnya, 100.000). Namun, Normal Equation dan SVD (Singular Value Decomposition, yang digunakan oleh `LinearRegression` Scikit-Learn) bersifat linear terhadap jumlah *instance* pelatihan ($O(m)$), sehingga efisien untuk *dataset* pelatihan besar selama mereka dapat masuk ke memori.

2.  **Gradient Descent (GD)**: Ini adalah pendekatan optimasi iteratif yang secara bertahap menyesuaikan parameter model untuk meminimalkan fungsi biaya.
    * **Ide Umum**: Algoritma ini mengukur gradien lokal dari fungsi *error* terhadap vektor parameter $\theta$ dan bergerak ke arah gradien yang menurun. Ketika gradien nol, minimum telah tercapai.
    * **Learning Rate ($\eta$)**: Ukuran langkah ditentukan oleh *learning rate*. Jika terlalu kecil, konvergensi akan lambat. Jika terlalu besar, algoritma dapat menyimpang.
    * **Jenis-jenis Gradient Descent**:
        * **Batch Gradient Descent**: Menghitung gradien berdasarkan seluruh *training set* pada setiap langkah. Ini lambat pada *training set* yang sangat besar, tetapi skalanya baik dengan jumlah fitur. Persamaan turunan parsial dari fungsi biaya terhadap parameter $\theta_j$ adalah $\frac{\partial}{\partial\theta_{j}}MSE(\theta)=\frac{2}{m}\sum_{i=1}^{m}(\theta^{T}x^{(i)}-y^{(i)})x_{j}^{(i)}$. Vektor gradiennya adalah $\nabla_{\theta}MSE(\theta)$. Langkah *update* parameter adalah $\theta^{(\text{next step})} = \theta - \eta \nabla_{\theta}MSE(\theta)$.
            * **Contoh Kode Batch GD**:
                ```python
                eta = 0.1 # learning rate
                n_iterations = 1000
                m = 100
                theta = np.random.randn(2,1) # inisialisasi random

                for iteration in range(n_iterations):
                    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
                    theta = theta - eta * gradients
                print(theta)
                ```
                Kode ini mengimplementasikan Batch Gradient Descent dan akan menghasilkan `theta` yang sangat mirip dengan yang ditemukan oleh Normal Equation.
        * **Stochastic Gradient Descent (SGD)**: Memilih *instance* acak dari *training set* pada setiap langkah dan menghitung gradien hanya berdasarkan *instance* tunggal itu. Ini jauh lebih cepat karena memanipulasi sangat sedikit data per iterasi dan memungkinkan pelatihan pada *training set* yang sangat besar (*out-of-core*). Namun, karena sifatnya yang stokastik, fungsi biaya akan memantul naik-turun dan tidak akan benar-benar menetap di minimum. Randomness ini dapat membantu keluar dari *local minima*. Untuk konvergensi yang lebih baik, *learning rate* harus dikurangi secara bertahap (*learning schedule*).
            * **Contoh Kode SGD**:
                ```python
                n_epochs = 50
                t0, t1 = 5, 50 # hyperparameters learning schedule

                def learning_schedule(t):
                    return t0 / (t + t1)

                theta = np.random.randn(2,1) # inisialisasi random

                for epoch in range(n_epochs):
                    for i in range(m):
                        random_index = np.random.randint(m)
                        xi = X_b[random_index:random_index+1]
                        yi = y[random_index:random_index+1]
                        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
                        eta = learning_schedule(epoch * m + i)
                        theta = theta - eta * gradients
                print(theta)
                ```
                Kode ini menunjukkan implementasi SGD dengan *learning schedule* sederhana.
        * **Mini-batch Gradient Descent**: Menghitung gradien pada kumpulan *instance* acak yang lebih kecil (*mini-batches*). Ini memanfaatkan optimasi perangkat keras untuk operasi matriks (terutama GPU) dan kemajuannya lebih stabil dibandingkan SGD, tetapi mungkin lebih sulit untuk keluar dari *local minima*.

**Polynomial Regression**
Polynomial Regression memungkinkan model linear untuk menyesuaikan data non-linear dengan menambahkan pangkat dari setiap fitur sebagai fitur baru.
* **Contoh Kode**:
    ```python
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.linear_model import LinearRegression
    import numpy as np

    m = 100
    X = 6 * np.random.rand(m, 1) - 3
    y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

    poly_features = PolynomialFeatures(degree=2, include_bias=False)
    X_poly = poly_features.fit_transform(X)
    print(X[0])
    print(X_poly[0])

    lin_reg = LinearRegression()
    lin_reg.fit(X_poly, y)
    print(lin_reg.intercept_, lin_reg.coef_)
    ```
    Kode ini menghasilkan data non-linear, menambahkan fitur kuadratnya (derajat 2) untuk membentuk `X_poly`, dan kemudian melatih model `LinearRegression` pada `X_poly`.

**Learning Curves**
*Learning curves* adalah plot kinerja model pada *training set* dan *validation set* sebagai fungsi dari ukuran *training set*. Ini membantu mendeteksi *overfitting* atau *underfitting*.
* **Underfitting**: Kedua kurva (pelatihan dan validasi) mencapai *plateau*, dekat satu sama lain, dan relatif tinggi. Menambahkan lebih banyak contoh pelatihan tidak akan membantu.
* **Overfitting**: *Error* pada data pelatihan jauh lebih rendah daripada *error* pada data validasi, menunjukkan adanya kesenjangan antara kedua kurva. Menambahkan lebih banyak data pelatihan dapat membantu mengurangi *overfitting*.

**Bias/Variance Trade-off**
*Generalization error* model dapat dipecah menjadi tiga komponen:
* **Bias**: Kesalahan karena asumsi yang salah (misalnya, mengasumsikan data linear padahal kuadratik). Model dengan *high-bias* cenderung *underfit*.
* **Variance**: Kesalahan karena sensitivitas berlebihan model terhadap variasi kecil dalam data pelatihan. Model dengan banyak *degrees of freedom* (misalnya, polinomial tingkat tinggi) cenderung memiliki *high variance* dan *overfit*.
* **Irreducible error**: Kesalahan karena *noise* dalam data itu sendiri.

Meningkatkan kompleksitas model biasanya meningkatkan *variance* dan mengurangi *bias*, dan sebaliknya. Ini disebut *trade-off*.

**Regularized Linear Models**
Regularisasi adalah cara untuk mengurangi *overfitting* dengan membatasi model. Untuk model linear, ini biasanya dicapai dengan membatasi bobot model.

1.  **Ridge Regression (L2 Regularization)**: Menambahkan *regularization term* $\alpha\frac{1}{2}\Sigma_{i=1}^{n}\theta_{i}^{2}$ ke fungsi biaya MSE. Ini memaksa algoritma pembelajaran untuk menjaga bobot model sekecil mungkin. *Hyperparameter* $\alpha$ mengontrol kekuatan regularisasi. $\theta_0$ (bias term) tidak diregularisasi. Penting untuk menskalakan data sebelum melakukan Ridge Regression.
    * **Contoh Kode**:
        ```python
        from sklearn.linear_model import Ridge
        ridge_reg = Ridge(alpha=1, solver="cholesky")
        ridge_reg.fit(X, y)
        print(ridge_reg.predict([[1.5]]))

        from sklearn.linear_model import SGDRegressor
        sgd_reg = SGDRegressor(penalty="l2") # Menggunakan l2 regularization
        sgd_reg.fit(X, y.ravel())
        print(sgd_reg.predict([[1.5]]))
        ```
        `Ridge` menggunakan solusi *closed-form*, sedangkan `SGDRegressor` dengan `penalty="l2"` menggunakan Gradient Descent dengan regularisasi L2.

2.  **Lasso Regression (L1 Regularization)**: Menambahkan *regularization term* $\alpha\Sigma_{i=1}^{n}|\theta_{i}|$ ke fungsi biaya. Ciri penting dari Lasso Regression adalah ia cenderung menghilangkan bobot fitur yang paling tidak penting (mengaturnya menjadi nol), sehingga melakukan seleksi fitur otomatis dan menghasilkan model yang *sparse*.
    * **Contoh Kode**:
        ```python
        from sklearn.linear_model import Lasso
        lasso_reg = Lasso(alpha=0.1)
        lasso_reg.fit(X, y)
        print(lasso_reg.predict([[1.5]]))
        ```
        Sama seperti Ridge, `SGDRegressor` juga dapat digunakan dengan `penalty="l1"`.

3.  **Elastic Net**: Gabungan antara Ridge dan Lasso Regression. *Regularization term* adalah campuran dari kedua istilah regularisasi tersebut, dikontrol oleh rasio campuran $r$. Ketika $r=0$, Elastic Net setara dengan Ridge Regression, dan ketika $r=1$, setara dengan Lasso Regression. Secara umum, Elastic Net lebih disukai daripada Lasso karena Lasso dapat berperilaku tidak menentu ketika jumlah fitur lebih besar dari jumlah *instance* pelatihan atau ketika beberapa fitur sangat berkorelasi.
    * **Contoh Kode**:
        ```python
        from sklearn.linear_model import ElasticNet
        elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5) # l1_ratio adalah r
        elastic_net.fit(X, y)
        print(elastic_net.predict([[1.5]]))
        ```

**Early Stopping**
*Early stopping* adalah cara lain untuk meregularisasi algoritma pembelajaran iteratif seperti Gradient Descent. Ini melibatkan penghentian pelatihan segera setelah *validation error* mencapai minimumnya. Ketika *error* validasi mulai naik, itu menunjukkan bahwa model telah mulai *overfit* data pelatihan.
* **Contoh Kode**:
    ```python
    from sklearn.base import clone
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import PolynomialFeatures, StandardScaler
    from sklearn.linear_model import SGDRegressor
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_squared_error
    import numpy as np

    # Data persiapan (contoh sederhana)
    X = 2 * np.random.rand(100, 1)
    y = 4 + 3 * X + np.random.randn(100, 1)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler())
    ])

    X_train_poly_scaled = poly_scaler.fit_transform(X_train)
    X_val_poly_scaled = poly_scaler.transform(X_val)

    sgd_reg = SGDRegressor(max_iter=1, tol=-np.infty, warm_start=True,
                           penalty=None, learning_rate="constant", eta0=0.0005, random_state=42)

    minimum_val_error = float("inf")
    best_epoch = None
    best_model = None

    for epoch in range(1000):
        sgd_reg.fit(X_train_poly_scaled, y_train.ravel()) # .ravel() needed for y
        y_val_predict = sgd_reg.predict(X_val_poly_scaled)
        val_error = mean_squared_error(y_val, y_val_predict)

        if val_error < minimum_val_error:
            minimum_val_error = val_error
            best_epoch = epoch
            best_model = clone(sgd_reg)
    print(f"Best epoch: {best_epoch}, Minimum validation error: {np.sqrt(minimum_val_error)}")
    # The best_model now holds the model state at the minimum validation error
    ```
    Dalam kode ini, `warm_start=True` memungkinkan metode `fit()` untuk melanjutkan pelatihan dari keadaan sebelumnya, bukan memulai dari awal.

**Logistic Regression**
Logistic Regression digunakan untuk memperkirakan probabilitas suatu *instance* termasuk dalam kelas tertentu (klasifikasi biner).
* **Estimasi Probabilitas**: Model menghitung jumlah fitur input yang diberi bobot (ditambah *bias term*), kemudian menghasilkan *logistic* (fungsi sigmoid) dari hasil ini. Fungsi *logistic* adalah $\sigma(t)=\frac{1}{1+exp(-t)}$.
* **Prediksi**: Jika probabilitas yang diperkirakan ($\hat{p}$) lebih besar dari 50%, model memprediksi *instance* tersebut termasuk dalam kelas positif (label "1"), jika tidak, kelas negatif (label "0").
* **Fungsi Biaya**: Fungsi biaya untuk satu *instance* pelatihan adalah $c(\theta)=\begin{cases}-log(\hat{p})&if~y=1\\ -log(1-\hat{p})if~y=0\end{cases}$. Untuk seluruh *training set*, fungsi biayanya adalah *log loss* (Cross Entropy) $J(\theta)=-\frac{1}{m}\Sigma_{i=1}^{m}[y^{(i)}log(\hat{p}^{(i)})+(1-y^{(i)})log(1-\hat{p}^{(i)})]$.
* **Pelatihan**: Tidak ada solusi *closed-form* (seperti Normal Equation) untuk Logistic Regression, tetapi fungsi biaya bersifat konveks, sehingga Gradient Descent dapat menemukan minimum global. Turunan parsial dari fungsi biaya adalah $\frac{\partial}{\partial\theta_{j}}J(\theta)=\frac{1}{m}\sum_{i=1}^{m}(\sigma(\theta^{T}x^{(i)})-y^{(i)})x_{j}^{(i)}$.
* **Decision Boundaries**: Logistic Regression menghasilkan *decision boundary* linear.
* **Regularisasi**: Scikit-Learn `LogisticRegression` secara default menambahkan penalti $l_2$. *Hyperparameter* `C` mengontrol kekuatan regularisasi, di mana nilai `C` yang lebih tinggi berarti lebih sedikit regularisasi.

**Softmax Regression (Multinomial Logistic Regression)**
Softmax Regression adalah generalisasi Logistic Regression untuk mendukung banyak kelas secara langsung.
* **Cara Kerja**: Model menghitung skor $s_k(x)$ untuk setiap kelas $k$ (mirip dengan prediksi Linear Regression: $s_k(x) = x^T\theta^{(k)}$). Kemudian, probabilitas setiap kelas diperkirakan dengan menerapkan fungsi *softmax* ke skor tersebut. Fungsi *softmax* adalah $\hat{P}_{k}=\sigma(s(x))_{k}=\frac{exp(s_{k}(x))}{\Sigma_{j=1}^{K}exp(s_{j}(x))}$.
* **Prediksi**: Klasifier Softmax Regression memprediksi kelas dengan probabilitas tertinggi (atau skor tertinggi). Ini hanya dapat digunakan dengan kelas-kelas yang saling eksklusif.
* **Fungsi Biaya**: Tujuannya adalah meminimalkan fungsi biaya *cross entropy* $J(\Theta)=-\frac{1}{m}\Sigma_{i=1}^{m}\Sigma_{k=1}^{K}y_{k}^{(i)}log(\hat{p}_{k}^{(i)})$. *Gradient vector* untuk kelas $k$ adalah $\nabla_{\theta^{(k)}}J(\Theta)=\frac{1}{m}\sum_{i=1}^{m}(\hat{p}_{k}^{(i)}-y_{k}^{(i)})x^{(i)}$.
* **Contoh Kode**:
    ```python
    from sklearn import datasets
    from sklearn.linear_model import LogisticRegression
    import numpy as np

    iris = datasets.load_iris()
    X = iris["data"][:, (2, 3)] # petal length, petal width
    y = iris["target"]

    softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10)
    softmax_reg.fit(X, y)

    print(softmax_reg.predict([[5, 2]]))
    print(softmax_reg.predict_proba([[5, 2]]))
    ```
    Ini melatih model Softmax Regression pada *iris dataset* untuk mengklasifikasikan bunga ke dalam tiga spesies. `multi_class="multinomial"` dan `solver="lbfgs"` diatur untuk mengaktifkan Softmax Regression.

**Latihan dan Penjelasan Teoritis**

1.  **Algoritma Pelatihan Linear Regression yang dapat digunakan dengan *training set* yang memiliki jutaan fitur?**
    * Algoritma yang cocok adalah **Gradient Descent (GD) variants** seperti **Batch Gradient Descent**, **Stochastic Gradient Descent (SGD)**, atau **Mini-batch Gradient Descent**.
    * **Penjelasan Teoritis**: Normal Equation dan SVD (digunakan oleh `LinearRegression` Scikit-Learn) melibatkan inversi matriks $X^TX$ yang memiliki kompleksitas komputasi $O(n^{2.4})$ hingga $O(n^3)$ di mana $n$ adalah jumlah fitur. Ini menjadi sangat lambat ketika $n$ besar (misalnya, 100.000). Sebaliknya, Gradient Descent skalanya baik dengan jumlah fitur; melatih model Linear Regression dengan ratusan ribu fitur jauh lebih cepat menggunakan Gradient Descent dibandingkan Normal Equation atau dekomposisi SVD. SGD dan Mini-batch GD bahkan memiliki dukungan *out-of-core*, yang berarti mereka dapat menangani *training set* yang terlalu besar untuk masuk ke memori.

2.  **Fitur dalam *training set* memiliki skala yang sangat berbeda. Algoritma mana yang mungkin menderita, dan bagaimana? Apa yang bisa dilakukan?**
    * **Algoritma yang Menderita**: **Gradient Descent (GD) variants** (Batch GD, Stochastic GD, Mini-batch GD) dan **Regularized Linear Models** (Ridge, Lasso, Elastic Net).
    * **Bagaimana Mereka Menderita**:
        * **Gradient Descent**: Jika fitur memiliki skala yang sangat berbeda, fungsi biaya akan memiliki bentuk seperti mangkuk yang sangat memanjang (oval), bukan mangkuk bundar. Ini menyebabkan Gradient Descent bergerak hampir tegak lurus dengan arah menuju minimum global pada awalnya, dan kemudian "berbaris" lama di lembah yang hampir datar, sehingga membutuhkan waktu sangat lama untuk konvergen.
        * **Regularized Linear Models (Ridge, Lasso, Elastic Net)**: Model-model ini sensitif terhadap skala fitur input. Karena *regularization term* menambahkan penalti berdasarkan ukuran bobot fitur ($\Sigma\theta_i^2$ atau $\Sigma|\theta_i|$), fitur dengan skala yang lebih besar secara inheren akan memiliki bobot yang cenderung lebih kecil (atau sebaliknya jika tidak diskalakan), sehingga regularisasi akan mempengaruhi fitur-fitur tersebut secara tidak proporsional.
    * **Apa yang Bisa Dilakukan**: Pastikan semua fitur memiliki skala yang serupa. Ini dapat dilakukan dengan menggunakan **Feature Scaling** teknik seperti **Standardization** (menggunakan `StandardScaler` Scikit-Learn) atau **Normalization**.

3.  **Dapatkah Gradient Descent terjebak di *local minimum* ketika melatih model Logistic Regression?**
    * **Tidak, tidak seharusnya**.
    * **Penjelasan Teoritis**: Fungsi biaya Logistic Regression (log loss) adalah **fungsi konveks**. Fungsi konveks memiliki properti penting bahwa mereka hanya memiliki satu minimum global dan tidak ada *local minima* lainnya. Oleh karena itu, Gradient Descent (dengan *learning rate* yang tidak terlalu besar dan waktu yang cukup lama) dijamin akan mendekati minimum global.

4.  **Apakah semua algoritma Gradient Descent mengarah pada model yang sama, asalkan Anda membiarkannya berjalan cukup lama?**
    * **Tidak selalu, tapi idealnya mendekati**.
    * **Penjelasan Teoritis**:
        * **Batch Gradient Descent**: Akan konvergen ke minimum global karena fungsi biaya Linear Regression adalah konveks dan halus. Jika dijalankan cukup lama dan dengan *learning rate* yang tepat, ia akan mencapai solusi optimal.
        * **Stochastic Gradient Descent (SGD) dan Mini-batch Gradient Descent**: Karena sifat stokastik mereka, fungsi biaya akan memantul naik-turun dan tidak akan sepenuhnya menetap di minimum. Mereka akan berakhir sangat dekat dengan minimum global, tetapi tidak akan pernah berhenti tepat di sana kecuali *learning rate* berkurang ke nol. Namun, jika *learning schedule* yang baik digunakan (yaitu, secara bertahap mengurangi *learning rate*), mereka dapat mendekati minimum global secara arbitrer.
        * **Perbedaan Utama**: SGD dan Mini-batch GD mungkin memiliki peluang lebih baik untuk keluar dari *local minima* pada fungsi biaya yang tidak konveks (yang bukan kasus untuk Linear/Logistic Regression). Jadi, meskipun untuk Linear/Logistic Regression mereka semua akan menemukan minimum global (atau sangat dekat dengannya), perilaku konvergensi dan lintasan mereka berbeda.

5.  **Misalkan Anda menggunakan Batch Gradient Descent dan Anda memplot *validation error* pada setiap *epoch*. Jika Anda melihat bahwa *validation error* secara konsisten naik, apa yang kemungkinan terjadi? Bagaimana Anda bisa memperbaikinya?**
    * **Apa yang Terjadi**: Jika *validation error* secara konsisten naik, kemungkinan besar **learning rate ($\eta$) terlalu tinggi**. Ini menyebabkan algoritma "melompati" minimum dan menyimpang, dengan nilai fungsi biaya yang semakin besar pada setiap langkah.
    * **Bagaimana Memperbaikinya**:
        * **Kurangi *learning rate*** ($\eta$). Anda bisa mencoba *grid search* untuk menemukan *learning rate* yang optimal.
        * Pastikan fitur-fitur diskalakan dengan benar, karena skala yang berbeda juga dapat memperburuk masalah konvergensi pada GD.
        * Jika model terlalu kompleks (misalnya, derajat polinomial terlalu tinggi), itu mungkin sudah *overfit* bahkan dengan *learning rate* yang tepat. Dalam kasus tersebut, **regularisasi** (Ridge, Lasso, Elastic Net) atau **mengurangi kompleksitas model** perlu dipertimbangkan.

6.  **Apakah ide yang baik untuk segera menghentikan Mini-batch Gradient Descent ketika *validation error* naik?**
    * **Tidak, itu bukan ide yang baik untuk segera menghentikannya**.
    * **Penjelasan Teoritis**: Dengan Stochastic Gradient Descent dan Mini-batch Gradient Descent, kurva *validation error* tidak semulus Batch Gradient Descent. Mereka akan **memantul naik dan turun** karena sifat stokastik algoritma. Jika Anda berhenti segera setelah *error* validasi naik untuk pertama kalinya, Anda mungkin berhenti terlalu dini dan tidak mencapai minimum global.
    * **Apa yang Seharusnya Dilakukan**: Solusi yang lebih baik untuk *early stopping* dengan SGD/Mini-batch GD adalah **berhenti hanya setelah *validation error* berada di atas minimum selama beberapa waktu** (misalnya, beberapa *epoch*). Ini memberi keyakinan bahwa model tidak akan berkinerja lebih baik lagi. Kemudian, **kembalikan parameter model ke titik di mana *validation error* adalah minimum**. Ini sering diimplementasikan dengan melacak *best validation error* yang diamati dan menyimpan parameter model yang sesuai.